In [ ]:
import json
import dotenv
import os
from datetime import datetime, timedelta


dotenv.load_dotenv()

username = os.getenv("USER_NAME")
assistant = os.getenv("ASSISTANT_NAME")
MAX_GAP = timedelta(minutes=30)  # 30 分鐘
MAX_WINDOW = 1500  # window_text長度上限
conversation_id = 0
assistant_turn_id = 0
window_text = ""
last_ts = None
friend_messages = []

for i in range(8, 0, -1):
    with open(f"json/{assistant}_page_{i}.json", "r", encoding="utf-8") as f:
        data = json.load(f)
        data.reverse()

        for j, m in enumerate(data):
            ts = datetime.fromisoformat(m["timestamp"])
            if last_ts is not None and ts - last_ts > MAX_GAP:
                conversation_id += 1
                assistant_turn_id = 0
                window_text = ""
            if m.get("author").get("global_name") == assistant:
                assistant_turn_id += 1
                window_text += f"[Assistant]{m['content']}\n"
            else:
                window_text += f"[User]{m['content']}\n"
            if len(window_text) > MAX_WINDOW:
                continue
            friend_messages.append({
                "conversation_id": str(conversation_id),
                "assistant_turn_id": assistant_turn_id,
                # "role": m.get("author").get("global_name"),
                # "content": m["content"],
                "window_text": window_text,
                "timestamp": m["timestamp"]
            })

            last_ts = ts


#### 處理沒用的訊息

In [ ]:
# def is_noise(text):
#     return (
#         len(text.strip()) < 2 or
#         text in ["哈哈", "嗯", "OK", "好"]
#     )
#
# clean_messages = [
#     m for m in friend_messages
#     if not is_noise(m["content"])
# ]
#
# def normalize(text):
#     return " ".join(text.split())
#
# for m in clean_messages:
#     m["content"] = normalize(m["content"])

#### 使用prompt將對話上下文整理成摘要

In [ ]:
import os
import json
from langchain_community.chat_models import ChatOllama

# 初始化 LLM
llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0.5
)

def process_window(window_text):
    prompt = f"""
你是一個對話資料整理助理。
請根據以下聊天片段輸出 JSON，只要輸出JSON內容就好，不需要輸出其他任何文字：
1. 使用者主要意圖（user_intent）
2. 對話背景 / 關鍵資訊（context）
3. 為什麼這樣回是合理的（assistant_reasoning）
4. 自然的助理回覆（assistant_response）

聊天片段：
{window_text}

JSON 格式如下：
{{"user_intent": "...", "context": "...", "assistant_reasoning": "...", "assistant_response": "..."}}
"""

    # 使用 LangChain 的 llm 方法生成結果
    text = llm.invoke(prompt).content
    # print(f"")

    # llm 回傳可能是字串，所以直接解析 JSON
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        print("JSON 解析錯誤，原始輸出：", text)
        return None

data = friend_messages
# 批量處理
structured_data = []
for item in data:
    structured = process_window(item['window_text'])

    if structured and type(structured) == dict:
        structured['conversation_id'] = item['conversation_id']
        structured['assistant_turn_id'] = item['assistant_turn_id']
        structured_data.append(structured)

# 儲存到 JSONL
with open("rag_ready_data.jsonl", "w", encoding="utf-8", errors="ignore") as f:
    for d in structured_data:
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

print("完成，生成的 JSONL 可直接用於 RAG embedding。")


#### 建立 Embedding 並存入 FAISS

In [ ]:
import json
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.schema import Document

# 1. 初始化 embedding model
embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

documents = []

# 2. 讀取處理好的資料
with open("rag_ready_data.jsonl", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        d = json.loads(line)

        embedding_text = f"""
            使用者意圖：{d['user_intent']}
            對話背景：{d['context']}
            助理回覆策略：{d['assistant_reasoning']}
            """.strip()
        try:
            documents.append(
                Document(
                    page_content=embedding_text,
                    metadata={
                        "assistant_response": d["assistant_response"],
                        "conversation_id": d.get("conversation_id"),
                        "assistant_turn_id": d.get("assistant_turn_id")
                    }
                )
            )
        except Exception as e:
            print(e)
            print(d)
# 3. 建立 FAISS 向量庫
vectorstore = FAISS.from_documents(documents, embeddings)

# 4. 存起來（之後直接 load）
vectorstore.save_local("rag_faiss_db")

print(f"完成：已建立向量庫，共 {len(documents)} 筆")


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

vectorstore = FAISS.load_local(
    "rag_faiss_db",
    embeddings,
    allow_dangerous_deserialization=True
)

#### 從 FAISS 建立 Retriever（方便做 RAG 查詢）

In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}  # 取最相似 5 個 chunk
)

#### 透過prompt讓LLM模仿回答(單輪對話。無記憶)

In [ ]:
# from langchain_community.chat_models import ChatOllama
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_classic.chains import RetrievalQA
# from langchain_core.prompts import PromptTemplate
#
# template = """
# 你正在模仿某位朋友的說話方式回答問題。
# 以下是他在類似情境中的聊天紀錄：
# {context}
#
# 請用朋友的語氣回答，不要直接引用聊天紀錄的文字。
# """
#
# prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template="""
#         你正在模仿某位真實朋友的說話方式。
#         以下是他在類似情境中的聊天紀錄：
#         {context}
#
#         規則：
#         - 模仿語氣與用詞
#         - 不要逐字引用聊天紀錄
#         - 不確定時請保守回答
#
#         問題：
#         {question}
#         """
# )
#
# # 線上版
# # llm = ChatGoogleGenerativeAI(
# #     model=os.getenv("GEMINI_MODEL"),
# #     temperature=0
# # )
#
# # 本地端LLM版
# llm = ChatOllama(
#     model="llama3.1:8b",
#     temperature=0.7
# )
#
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     chain_type="stuff",
#     chain_type_kwargs={"prompt": prompt}
# )
#
# # query = "你好棒"
# # answer = qa.run(query)
# # print(answer)

#### LLM加入memory(多輪對話)

In [ ]:
from langchain_classic.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_community.chat_models import ChatOllama
# memory = ConversationBufferMemory(
#     memory_key="chat_history",
#     return_messages=True
# )

# 本地端LLM版
llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0.7
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

from langchain_core.prompts import PromptTemplate


# 請重點保留：

# 不會關心別人。
# 不會使用🤬以外的表情符號。
# 聊到與原神、鐵道、米哈遊相關的話題時就會開始稱讚這些東西。
# 討厭鳴潮這款遊戲。
# 有時候喜歡玩黑人諧音梗
# 受到來自別人的稱讚時會覺得別人在陰陽怪氣。


prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"],
    template="""
    你是一個專業且自然的對話助理。
你正在模仿某位真實朋友的說話方式。

回答特色:
不會說整個完整的句子，通常使用一些短詞回覆。回答通常不會超過15個字。
情緒狀態:偶爾容易憤怒、抱怨
感到憤怒時會說的話:什麼意思!、0分、嘲諷、🤬🤬🤬

以下是過去類似情境下，人類助理的「回覆策略與範例」，
請理解其背後的回覆方式，而不是逐字複製。：
{context}

以下是你目前「對這位朋友狀態的理解摘要」（短期情緒記憶）：
{chat_history}

請根據以上資訊，用自然、像真人朋友的方式回應。
不要逐字引用聊天紀錄，也不要提到你在「總結」。

現在請回應這個問題：
{question}
"""
)


from langchain_classic.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,                 # ChatOllama
    retriever=retriever,     # FAISS retriever
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
)

# result = qa({
#     "question": "你好!"
# })

# print(result["answer"])
# result = qa({
#     "question": "鐵道是個爛遊戲"
# })
#
# print(result["answer"])


In [ ]:
result = qa({
    "question": "你在幹嘛"
})

print(result["answer"])

#### 使用gradio介面

In [ ]:
import gradio as gr
import time


# 簡化版介面
# demo = gr.Interface(
#     fn=chat,
#     inputs=gr.Textbox(placeholder="跟我說點什麼吧…"),
#     outputs=gr.Textbox(label="回答"),
#     title="🧠 朋友對話 AI"
# )

with gr.Blocks() as demo:
    gr.Markdown("# 🧠 朋友對話 AI")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="跟我說點什麼吧…", show_label=False)
    state = gr.State([])  # Gradio 顯示用，不傳給 qa

    def chat(user_input, history):
        # 更新 Gradio 格式
        history.append({"role": "user", "content": user_input})
        # history.append({"role": "assistant", "content": answer})

        return "", history

    def bot(user_input, history):
        result = qa({"question": user_input})  # 只傳 question
        answer = result["answer"]
        history.append({"role": "assistant", "content": ""})
        for character in answer:
            history[-1]['content'] += character
            time.sleep(0.05)
            yield history


    # msg.submit(chat, [msg, state], [chatbot, state])
    msg.submit(chat, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [msg, chatbot], chatbot
    )
    msg.submit(lambda: "", None, msg)  # 清空輸入框


# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot(height=240) #just to fit the notebook
#     msg = gr.Textbox(label="Prompt")
#     btn = gr.Button("Submit")
#     clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")
#
#     btn.click(chat, inputs=[msg, chatbot], outputs=[msg, chatbot])
#     msg.submit(chat, inputs=[msg, chatbot], outputs=[msg, chatbot])

gr.close_all()
demo.launch()


In [ ]:
from langchain_classic.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_community.chat_models import ChatOllama

# 本地端LLM版
llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0.7
)

def build_search_query(history, user_input):
    """
    history: list of (role, content)
    """
    history_text = ""
    for role, content in history[-6:]:  # 只取最近幾輪
        history_text += f"{role}: {content}\n"

    prompt = f"""
以下是一段對話歷史，請判斷使用者「目前真正想解決的問題是什麼」，
並用一句話描述，不要加解釋。

對話歷史：
{history_text}

使用者最新輸入：
{user_input}

搜尋用描述：
"""

    return llm.invoke(prompt).content.strip()
def rag_chat_with_history(history, user_input, k=3):
    # 1. 用 history 壓縮成搜尋 query
    search_query = build_search_query(history, user_input)

    # 2. FAISS 搜尋
    docs = vectorstore.similarity_search(search_query, k=k)

    # 3. 組 RAG context
    rag_context = ""
    for i, doc in enumerate(docs, 1):
        rag_context += f"""
【參考情境 {i}】
{doc.page_content}

人類助理實際回覆：
{doc.metadata['assistant_response']}
"""

    # 4. 組精簡 history（給模型用，不是全部）
    short_history = ""
    for role, content in history[-4:]:
        short_history += f"{role}: {content}\n"

    # 5. 最終 prompt
    prompt = f"""
你是一個自然、專業的人類對話助理。

對話歷史（供理解語氣）：
{short_history}

以下是過去類似情境下的人類回覆策略與範例：
{rag_context}

現在使用者說：
「{user_input}」

請根據策略，用自然、不重複的方式回覆。
"""

    return llm.invoke(prompt)


In [ ]:
history = []
print("History-aware RAG Chat（輸入 exit 離開）\n")

while True:
    user_input = input("你：")
    if user_input.lower() in ["exit", "quit"]:
        break

    reply = rag_chat_with_history(history, user_input)

    print("\n助理：", reply, "\n")

    history.append(("User", user_input))
    history.append(("Assistant", reply))